In [50]:
%load_ext autoreload
%autoreload 2

In [95]:
import os
import torch
from visualize._utils import draw_connectome
from utils import ROOT_DIR
from torch_geometric.data import Data
from tests.leandro.plots import *
from data._main import get_dataset
from omegaconf import OmegaConf

In [64]:
# load the connectome data
graph_tensors = torch.load(
    os.path.join(ROOT_DIR, "data", "processed", "connectome", "graph_tensors.pt")
)

# make the graph
graph = Data(**graph_tensors)

print('Graph attributes: {}'.format(graph.keys))
print('Num. nodes: {}, Num. edges: {}, Num. node features: {}'.format(graph.num_nodes, graph.num_edges, graph.num_node_features))

Graph attributes: ['n_id', 'y', 'node_type', 'pos', 'edge_index', 'num_classes', 'edge_attr', 'idx_to_neuron', 'x']
Num. nodes: 302, Num. edges: 4396, Num. node features: 1024


In [81]:
neuron_to_idx = {u: v for (v, u) in graph.idx_to_neuron.items()}

In [93]:
def get_connected_neuron_indexes(graph, neuron_name):

    neuron_to_idx = {u: v for (v, u) in graph.idx_to_neuron.items()}

    # Get the edge indices corresponding to the given node
    node_index = neuron_to_idx[neuron_name]
    connected_edge_indices = (graph.edge_index[0] == node_index).nonzero()

    # Get the connected nodes by extracting the second row of the connected_edge_indices
    connected_nodes = graph.edge_index[1, connected_edge_indices].flatten()
    
    return connected_nodes

print(graph.edge_index.shape) # Row 0: source, Row 1: target

print(graph.edge_attr.shape) # Column 0: gap junction connections, Column 1: chemical synapse connections

torch.Size([2, 4396])
torch.Size([4396, 2])


In [76]:
connected_neurons = get_connected_neuron_indexes(graph, 'ADAL')
connected_neurons

tensor([  0,   1,   6,  10,  12,  13,  53,  54,  55,  56,  57,  58,  59,  67,
        155, 179, 191, 192, 199, 201, 247, 250, 254])

In [96]:
config = OmegaConf.load("/home/lrvnc/Projects/worm-graph/conf/dataset.yaml")
flavell_dataset = get_dataset(config)

Chosen dataset(s): ['Flavell2023']
Num. worms: 50



In [106]:
def get_one_worm_data(dataset):
    wormid = np.random.choice([key for key in dataset.keys()])
    oneWorm = dataset[wormid]
    print('Worm ID: {}'.format(wormid))
    calcium_data = oneWorm['calcium_data']
    time_vector = oneWorm['time_in_seconds']
    return oneWorm, calcium_data, time_vector

oneWorm, calcium_data, time_vector = get_one_worm_data(flavell_dataset)

Worm ID: worm48
